In [36]:
import pynput
import mediapipe as mp
import cv2
import numpy as np

In [57]:
import cv2
import numpy as np
import mediapipe as mp
import pynput
import time

# Initialize MediaPipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
pose = mp_pose.Pose(static_image_mode=False)

# Initialize keyboard and mouse controllers
keyboard = pynput.keyboard.Controller()
mouse = pynput.mouse.Controller()

# State to control recording of events
record_events = False
countdown = 0  # Variable to manage countdown timer
countdown_started = False  # Flag to track if countdown is running

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ab = a - b
    cb = c - b

    dot_product = np.dot(ab, cb)
    magnitude_ab = np.linalg.norm(ab)
    magnitude_cb = np.linalg.norm(cb)

    angle = np.arccos(dot_product / (magnitude_ab * magnitude_cb))
    return np.degrees(angle)

# Function to get the color based on angle for visual feedback
def get_angle_color(angle):
    if angle < 45:
        return (0, 255, 0)  # Green for low angle (active gesture)
    elif 45 <= angle <= 90:
        return (255, 255, 0)  # Yellow for neutral
    else:
        return (0, 0, 255)  # Red for high angle (inactive)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks is not None:
        # Draw landmarks on the frame
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(128, 0, 250), thickness=2, circle_radius=3),
            mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
        )

        frame = cv2.flip(frame, 1)
        height, width, _ = frame.shape

        landmarks = results.pose_landmarks.landmark

        # Extract coordinates for various body landmarks
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW]
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]

        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
        left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]

        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]
        right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE]
        right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]

        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
        left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE]
        left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]

        nose = landmarks[mp_pose.PoseLandmark.NOSE]

        # Calculate body part coordinates
        right_shoulder_coords = (right_shoulder.x * width, right_shoulder.y * height)
        right_elbow_coords = (right_elbow.x * width, right_elbow.y * height)
        right_wrist_coords = (right_wrist.x * width, right_wrist.y * height)

        left_shoulder_coords = (left_shoulder.x * width, left_shoulder.y * height)
        left_elbow_coords = (left_elbow.x * width, left_elbow.y * height)
        left_wrist_coords = (left_wrist.x * width, left_wrist.y * height)

        right_hip_coords = (right_hip.x * width, right_hip.y * height)
        right_knee_coords = (right_knee.x * width, right_knee.y * height)
        right_ankle_coords = (right_ankle.x * width, right_ankle.y * height)

        left_hip_coords = (left_hip.x * width, left_hip.y * height)
        left_knee_coords = (left_knee.x * width, left_knee.y * height)
        left_ankle_coords = (left_ankle.x * width, left_ankle.y * height)

        nose_coords = (nose.x * width, nose.y * height)

        # Calculate angles for arms, legs, and nose
        right_arm_angle = calculate_angle(right_shoulder_coords, right_elbow_coords, right_wrist_coords)
        left_arm_angle = calculate_angle(left_shoulder_coords, left_elbow_coords, left_wrist_coords)

        right_leg_angle = calculate_angle(right_hip_coords, right_knee_coords, right_ankle_coords)
        left_leg_angle = calculate_angle(left_hip_coords, left_knee_coords, left_ankle_coords)

        nose_angle = calculate_angle(left_shoulder_coords, nose_coords, right_shoulder_coords)

        shoulder_center_x = (right_shoulder_coords[0] + left_shoulder_coords[0]) / 2

        # Display angle information with improved visuals
        cv2.rectangle(frame, (10, 30), (250, 60), (0, 0, 0), thickness=cv2.FILLED)
        cv2.putText(frame, f"Angulo Brazo Derecho: {int(right_arm_angle)}", (10, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, get_angle_color(right_arm_angle), 2)

        cv2.rectangle(frame, (10, 50), (250, 80), (0, 0, 0), thickness=cv2.FILLED)
        cv2.putText(frame, f"Angulo Brazo Izquierdo: {int(left_arm_angle)}", (10, 70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, get_angle_color(left_arm_angle), 2)

        cv2.rectangle(frame, (10, 70), (250, 100), (0, 0, 0), thickness=cv2.FILLED)
        cv2.putText(frame, f"Angulo Pierna Derecha: {int(right_leg_angle)}", (10, 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, get_angle_color(right_leg_angle), 2)

        cv2.rectangle(frame, (10, 90), (250, 120), (0, 0, 0), thickness=cv2.FILLED)
        cv2.putText(frame, f"Angulo Pierna Izquierda: {int(left_leg_angle)}", (10, 110), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, get_angle_color(left_leg_angle), 2)

        # Mouse simulation with visual feedback
        if record_events:
            if right_arm_angle <= 45 and left_arm_angle <= 45:
                # When both arms are below 45, press "w" and do not click
                keyboard.press("w")
                mouse.release(pynput.mouse.Button.left)
                mouse.release(pynput.mouse.Button.right)
            else:
                keyboard.release("w")
                # Handle mouse clicks based on individual arm angles
                if right_arm_angle <= 45:
                    mouse.press(pynput.mouse.Button.left)
                else:
                    mouse.release(pynput.mouse.Button.left)

                if left_arm_angle <= 45:
                    mouse.press(pynput.mouse.Button.right)
                else:
                    mouse.release(pynput.mouse.Button.right)

            if nose_coords[0] > shoulder_center_x + 10:
                mouse.move(-40, 0)
            elif nose_coords[0] < shoulder_center_x - 10:
                mouse.move(40, 0)

            if nose_angle < 70:
                mouse.move(0, -40)
            elif nose_angle > 80:
                mouse.move(0, 40)
            
            if right_leg_angle <= 60:
                keyboard.press(pynput.keyboard.Key.space)
            else:
                keyboard.release(pynput.keyboard.Key.space)
            
            if left_leg_angle <= 60:
                keyboard.press(pynput.keyboard.Key.shift_l)
            else:
                keyboard.release(pynput.keyboard.Key.shift_l)
        
        # Handle countdown for Enter key press
        if cv2.waitKey(1) & 0xFF == 13 and not countdown_started:
            countdown = 5  # Start countdown
            countdown_started = True

        # Display countdown if it is active
        if countdown_started:
            cv2.putText(frame, f"Activando: {countdown}", (width - 250, height - 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            if countdown > 0:
                countdown -= 1
                time.sleep(1)
            else:
                record_events = not record_events  # Toggle recording state
                countdown_started = False  # Reset countdown flag

    cv2.imshow("Frame", frame)

    # Break loop when the Escape key is pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 